In [248]:
import os
import requests
import zipfile
import pandas as pd
import sqlalchemy as sa
from datetime import datetime

In [249]:
url = 'https://leandrolessa.com.br/wp-content/uploads/2024/04/processamento_licencas_medicas.zip'
 
response = requests.get(url)

In [250]:

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

response = requests.get(url, headers=headers)

diretorio_destino = '/home/naum/studies/bases/processamento_dados_medicos'

# Verifica se o diretório de destino existe, se não, cria-o
if not os.path.exists(diretorio_destino):
    os.makedirs(diretorio_destino)

if response.status_code == requests.codes.OK:
    nome_arquivo_zip = url.split('/')[-1]
    caminho_arquivo_zip = os.path.join(diretorio_destino, nome_arquivo_zip)

    with open(caminho_arquivo_zip, 'wb') as arquivo:
        arquivo.write(response.content)
        print("Download finalizado. Salvo em: {}".format(caminho_arquivo_zip))
else:
    print("Falha ao baixar o arquivo. Status code:", response.status_code)


Download finalizado. Salvo em: /home/naum/studies/bases/processamento_dados_medicos/processamento_licencas_medicas.zip


In [251]:
with zipfile.ZipFile(caminho_arquivo_zip, 'r') as zip_ref:
        zip_ref.extractall(diretorio_destino)
        os.remove(caminho_arquivo_zip)

print("Arquivo ZIP removido.") 
print(f"Arquivo ZIP descompactado em: {diretorio_destino}")

Arquivo ZIP removido.
Arquivo ZIP descompactado em: /home/naum/studies/bases/processamento_dados_medicos


In [252]:
PATH = '/home/naum/studies/bases/processamento_dados_medicos'
lista_path = os.listdir(PATH)

In [253]:
lista_df = [] # cria uma lista vazia
for arquivo in lista_path:
    local_arquivo = os.path.join(PATH, arquivo)
    if arquivo.endswith('.csv'):
        df = pd.read_csv(local_arquivo, sep = '|', encoding='latin-1')       
        lista_df.append(df)

In [254]:
df = pd.concat(lista_df)

# Converta a coluna 'data_nascimento' para o tipo datetime se ainda não estiver
df['data_nascimento'] = pd.to_datetime(df['data_nascimento'])

# Calcule a idade
df['idade'] = (pd.to_datetime('today').year - df['data_nascimento'].dt.year)

print(df)

   data_processamento   nome_medico           especialidade  \
0          2022-10-09     Dr. Ramos             Ortopedista   
1          2022-10-09  Dra. Daniele         Traumatologista   
2          2022-10-09  Dra. Daniele         Traumatologista   
0          2023-11-25    Dr. Irineu  Ginecologista obstetra   
0          2023-10-10   Dr. Gustavo           Clínico geral   
..                ...           ...                     ...   
0          2023-02-15   Dr. Rodrigo             Ortopedista   
0          2022-11-02  Dr. Vinicius         Traumatologista   
0          2022-06-23   Dr. Gustavo           Clínico geral   
1          2022-06-23   Dr. Gustavo           Clínico geral   
2          2022-06-23      Dra. Ana  Ginecologista obstetra   

                    motivo_licenca  id_colaborador      nome_colaborador  \
0   Enfermidade de filhos pequenos             337    Maria Eduarda Lima   
1                            Comum             677         Alana Cardoso   
2              

In [268]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Inicialize a sessão Spark
spark = SparkSession.builder \
    .appName("dados_medicos") \
    .getOrCreate()

# Supondo que df seja o DataFrame que você já possui

# Crie o DataFrame Spark
df_spark = spark.createDataFrame(df)

# Converta a coluna 'data_nascimento' para o tipo de dados 'string'
df_spark = df_spark.withColumn('data_nascimento_str', F.col('data_nascimento').cast('string'))

# Extraia apenas a parte da data (sem a parte do tempo)
df_spark = df_spark.withColumn('data_nascimento', substring('data_nascimento_str', 1, 10))

# Converta a coluna 'data_nascimento' para o tipo de dados 'date'
df_spark = df_spark.withColumn('data_nascimento', to_date('data_nascimento', 'yyyy-MM-dd'))

# Agora podemos remover a coluna auxiliar 'data_nascimento_str' se desejado
df_spark = df_spark.drop('data_nascimento_str')

# Imprima o schema e os dados para verificar a mudança
df_spark.printSchema()
df_spark.show()


root
 |-- data_processamento: string (nullable = true)
 |-- nome_medico: string (nullable = true)
 |-- especialidade: string (nullable = true)
 |-- motivo_licenca: string (nullable = true)
 |-- id_colaborador: long (nullable = true)
 |-- nome_colaborador: string (nullable = true)
 |-- data_nascimento: date (nullable = true)
 |-- estado_colaborador: string (nullable = true)
 |-- sexo_colaborador: string (nullable = true)
 |-- inicio_licenca: string (nullable = true)
 |-- fim_licenca: string (nullable = true)
 |-- duracao_licenca: long (nullable = true)
 |-- escolaridade: string (nullable = true)
 |-- salario: double (nullable = true)
 |-- qtd_filhos: double (nullable = true)
 |-- hobbie: string (nullable = true)
 |-- possui_pet: string (nullable = true)
 |-- estado_civil: string (nullable = true)
 |-- idade: long (nullable = true)

+------------------+------------+--------------------+--------------------+--------------+------------------+---------------+-------------------+------------

In [280]:
from pyspark.sql import functions as F

# Supondo que df_spark seja o DataFrame que você já possui

# Calcula a idade dos colaboradores considerando o dia 18/04/2024
df_spark = df_spark.withColumn("idade", F.floor(F.datediff(F.lit("2024-04-19"), "data_nascimento") / 365))

# Filtra os colaboradores do sexo masculino e solteiros(as)
subset = (
    df_spark
    .filter(
        (F.col('sexo_colaborador') == 'Masculino') &
        (F.col('estado_civil') == 'Solteiro(a)')
    )
    .select('nome_colaborador', 'sexo_colaborador', 'data_nascimento', 'idade', 'estado_civil')
)

# Calcula a média da idade
media_idade = subset.agg(F.avg("idade")).collect()[0][0]

# Exibe apenas o valor da média no output
print("Média da idade:", media_idade)


Média da idade: 44.436974789915965


In [282]:
from pyspark.sql import functions as F

# Filtrar colaboradores com mais de 35 anos
df_filtered = df_spark.filter(F.col('idade') > 35)

# Calcular o valor mais repetido na coluna 'motivo_licenca'
most_common_reason = (
    df_filtered
    .groupBy('motivo_licenca')
    .count()
    .orderBy(F.desc('count'))
    .select('motivo_licenca')
    .limit(1)
).collect()[0][0]

# Exibir o valor mais repetido
print("Valor mais repetido na coluna motivo_licenca para colaboradores com mais de 35 anos:", most_common_reason)


Valor mais repetido na coluna motivo_licenca para colaboradores com mais de 35 anos: Comum


In [284]:
from pyspark.sql import functions as F

# Filtrar colaboradores do estado de Minas Gerais
df_mg = df_spark.filter(F.col('estado_colaborador') == 'Minas Gerais')

# Contar o número de colaboradores atendidos por médicos
num_atendidos_por_medico = (
    df_mg
    .filter(F.col('nome_medico').isNotNull())
    .groupBy('nome_medico')
    .count()
    .orderBy(F.desc('count'))
    .limit(1)
)

# Exibir a contagem de colaboradores atendidos por médico
num_atendidos_por_medico.show()


+-----------+-----+
|nome_medico|count|
+-----------+-----+
|  Dr. Pedro|    6|
+-----------+-----+



In [285]:
from pyspark.sql import functions as F

# Filtrar pessoas com ensino fundamental e leitura como hobbie
df_filtered = df_spark.filter((F.col('escolaridade') == 'Fundamental') & (F.col('hobbie') == 'Leitura'))

# Agrupar por estado e contar o número de pessoas em cada estado
count_by_state = df_filtered.groupBy('estado_colaborador').count()

# Encontrar o estado com o maior número de pessoas
max_count_state = count_by_state.orderBy(F.desc('count')).select('estado_colaborador').first()[0]

# Exibir o estado com o maior número de pessoas
print("Estado com o maior número de pessoas com ensino fundamental que têm leitura como hobbie:", max_count_state)


Estado com o maior número de pessoas com ensino fundamental que têm leitura como hobbie: Santa Catarina


In [287]:
from pyspark.sql import functions as F

# Filtrar pessoas com ensino superior e que são naturais ou pertencem a Pernambuco
df_filtered = df_spark.filter((F.col('escolaridade') == 'Superior') & ((F.col('estado_colaborador') == 'Pernambuco') | (F.col('estado_colaborador') == 'Pernambuco')))

# Contar o número de pessoas
count_people = df_filtered.count()

# Exibir o número de pessoas
print("Número de pessoas com ensino superior que são naturais ou pertencem a Pernambuco:", count_people)


Número de pessoas com ensino superior que são naturais ou pertencem a Pernambuco: 10


In [289]:
from pyspark.sql import functions as F

# Filtrar pessoas atendidas pelo Dr. Lessa
df_lessa = df_spark.filter(F.col('nome_medico') == 'Dr. Lessa')

# Calcular a média de filhos por sexo
avg_filhos_por_sexo = (
    df_lessa
    .groupBy('sexo_colaborador')
    .agg(F.avg('qtd_filhos').alias('media_filhos'))
)

# Exibir o nome do Dr. Lessa, o sexo do colaborador e a média de filhos para cada sexo
avg_filhos_por_sexo.show()


+----------------+-----------------+
|sexo_colaborador|     media_filhos|
+----------------+-----------------+
|        Feminino|2.380952380952381|
|       Masculino|              2.7|
+----------------+-----------------+



In [296]:
from pyspark.sql import functions as F

# Tratar os valores ausentes substituindo por zero
df_spark = df_spark.dropna(subset=['salario'])

# Encontre o colaborador com o maior salário
colaborador_maior_salario = (
    df_spark
    .orderBy(F.desc('salario'))
    .select('nome_colaborador', 'salario')
    .first()
)

# Exiba o colaborador com o maior salário
print("Colaborador com o maior salário:")
print("Nome:", colaborador_maior_salario['nome_colaborador'])
print("Salário:", colaborador_maior_salario['salario'])


Colaborador com o maior salário:
Nome: Nathan Barros
Salário: 28543.95


In [297]:
from pyspark.sql import functions as F

# Agrupar por motivo de licença e calcular a média de duração
media_duracao_licenca = (
    df_spark
    .groupBy('motivo_licenca')
    .agg(F.avg('duracao_licenca').alias('media_duracao'))
)

# Encontrar a licença com a menor média de duração
licenca_menor_media_duracao = (
    media_duracao_licenca
    .orderBy('media_duracao')
    .select('motivo_licenca', 'media_duracao')
    .first()
)

# Exibir a licença com a menor média de duração
print("Licença com a menor média de duração:")
print("Motivo da Licença:", licenca_menor_media_duracao['motivo_licenca'])
print("Média de Duração:", licenca_menor_media_duracao['media_duracao'])


Licença com a menor média de duração:
Motivo da Licença: Acidente de trabalho
Média de Duração: 14.96078431372549


In [298]:
from pyspark.sql import functions as F

# Agrupar por especialidade e contar o número de atendimentos
num_atendimentos_por_especialidade = (
    df_spark
    .groupBy('especialidade')
    .agg(F.count('*').alias('num_atendimentos'))
)

# Encontrar a especialidade com o menor número de atendimentos
especialidade_menor_atendimentos = (
    num_atendimentos_por_especialidade
    .orderBy('num_atendimentos')
    .select('especialidade', 'num_atendimentos')
    .first()
)

# Exibir a especialidade com o menor número de atendimentos
print("Especialidade com o menor número de atendimentos:")
print("Especialidade:", especialidade_menor_atendimentos['especialidade'])
print("Número de Atendimentos:", especialidade_menor_atendimentos['num_atendimentos'])



Especialidade com o menor número de atendimentos:
Especialidade: Ginecologista obstetra
Número de Atendimentos: 151


In [299]:
from pyspark.sql import functions as F

# Tratar os valores ausentes substituindo por zero
df_spark = df_spark.dropna(subset=['salario'])

# Agrupar por estado e calcular a média salarial
media_salarial_por_estado = (
    df_spark
    .groupBy('estado_colaborador')
    .agg(F.avg('salario').alias('media_salarial'))
)

# Encontrar o estado com a maior média salarial
estado_maior_media_salarial = (
    media_salarial_por_estado
    .orderBy(F.desc('media_salarial'))
    .select('estado_colaborador', 'media_salarial')

)

estado_maior_media_salarial.show()


+-------------------+------------------+
| estado_colaborador|    media_salarial|
+-------------------+------------------+
|           Maranhão| 6086.075609756099|
|       Minas Gerais| 6025.771891891892|
|               Pará| 6014.618571428572|
|            Roraima| 5933.727297297298|
|              Goiás| 5911.890555555557|
|              Amapá| 5809.281666666667|
|              Piauí| 5803.761282051282|
|           Amazonas| 5713.387209302325|
|              Ceará| 5707.987647058824|
|        Mato Grosso|          5582.535|
|          Tocantins| 5515.493863636364|
|Rio Grande do Norte| 5454.649142857142|
|     Santa Catarina| 5452.204444444444|
|              Bahia| 5432.883548387096|
|     Rio de Janeiro| 5413.823055555556|
|            Paraíba|  5402.67756097561|
|         Pernambuco|  5346.71606060606|
|           Rondônia| 5345.994210526315|
|            Alagoas|5230.0280555555555|
|          São Paulo| 5213.783055555556|
+-------------------+------------------+
only showing top